In [ ]:
%load_ext autoreload
%autoreload 2

import numpy as np
%matplotlib inline
import torch
from torch import nn, einsum
import torch.nn.functional as F

from skimage import measure
import matplotlib.pyplot as plt
import plotly
import plotly.graph_objs as go


from ddpm import *
from viz_x import viz_x

device = "cuda"

In [ ]:
from core.models import get_model
from dataset import get_dataset
from init import get_cfg, setup_seed, dev_get_cfg

# preparer configuration
category = "mugs"
# category = "kit4cates"
# category = "chairs"
cfg  =dev_get_cfg(category)
# prepare models
ModelClass = get_model(cfg["model"]["model_name"])
model = ModelClass(cfg)
ckpt_path = f"/home/ziran/se3/EFEM/weights/{category}.pt"
# ckpt_path = f"/home/ziran/se3/EFEM/lib_shape_prior/log/12_10_shape_prior_mugs_old/12_10_shape_prior_mugs_FOR_hopefullybetterAE/checkpoint/15409.pt"
ckpt = torch.load(ckpt_path)
model.network.load_state_dict(ckpt['model_state_dict'])
model.network.to(device)

# prepare dataset
DatasetClass = get_dataset(cfg)
datasets_dict = dict()
for mode in cfg["modes"]:
    datasets_dict[mode] = DatasetClass(cfg, mode=mode)

train_ds = datasets_dict["train"]

In [ ]:
codebook_path = f"/home/ziran/se3/EFEM/cache/mugs.npz"
# codebook_path = f"/home/ziran/se3/EFEM/lib_shape_prior/log/12_10_shape_prior_mugs_old/12_10_shape_prior_mugs_FOR_hopefullybetterAE/codebook.npz"
train_ds = CustomDataset(codebook_path)


In [ ]:
import random

idx_1, idx_2 = random.randint(0, 148), random.randint(0, 148)
print(idx_1, idx_2)
x_1 = train_ds[idx_1][0].to(device)
x_2 = train_ds[idx_2][0].to(device)

bs = 5
inter_range_list = np.linspace(0, 1, bs)
print(inter_range_list)
x_inter_list = [(1-alpha) * x_1+ alpha * x_2 for alpha in inter_range_list]
x_inter = torch.stack(x_inter_list, dim=0)
print(x_inter.shape)

In [ ]:
viz_x(x_inter[:,:,:], model, device)

In [ ]:
from_codebook = torch.stack([train_ds[idx][0] for idx in range(10)], dim=0).to(device)
viz_x(from_codebook, model)

In [ ]:
import random

idx_1, idx_2, idx_3 = random.randint(0, 148), random.randint(0, 148), random.randint(0, 148)
print(idx_1, idx_2, idx_3)
x_1 = train_ds[idx_1][0].to(device)
x_2 = train_ds[idx_2][0].to(device)
x_3 = train_ds[idx_3][0].to(device)

bs = 10
inter_range_list = np.linspace(0, 1, bs)
print(inter_range_list)
x_inter_list = [(1-alpha) * x_1+ alpha/2 * x_2+ alpha/2*x_3 for alpha in inter_range_list]
x_inter = torch.stack(x_inter_list, dim=0)
print(x_inter.shape)


In [ ]:
viz_x(x_inter, model)